# **Tarea 1/Opcinonal 1**: Transformar a RDF



## **Primer Conjunto de Datos (JSON):** "Cantidad de pasajeros embarcados por bloque horario en el Aeropuerto de Punta Arenas Presidente Carlos Ibáñez del Campo"

## Script, Generación en formato RDF, archivo ttl

In [ ]:
!pip install rdflib

In [ ]:
import json
import rdflib
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD
from google.colab import files

# Ruta del archivo JSON en Google Colab
json_filepath = "/content/sample_data/PasajerosembarcadosporhoraenelAeropuertoSCCI_2818101738163526694.json"

# Leer el archivo JSON
with open(json_filepath, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extraer encabezados y datos
headers = data[0]  # Primera fila contiene los nombres de las columnas
rows = data[1:]    # Filas de datos

# Crear un grafo RDF
g = Graph()

# Definir un namespace personalizado
EX = Namespace("http://example.org/airport#")
g.bind("ex", EX)

# Función para convertir fecha de 'DD-MM-YYYY' a 'YYYY-MM-DD'
def convertir_fecha(fecha_str):
    partes = fecha_str.split("-")  # Separar por guion
    return f"{partes[2]}-{partes[1]}-{partes[0]}"  # Reordenar como YYYY-MM-DD

# Iterar sobre los datos y agregarlos al grafo
for i, row in enumerate(rows):
    registro_uri = URIRef(EX[f"registro_{i}"])

    # Agregar tipo de entidad
    g.add((registro_uri, RDF.type, EX.RegistroEmbarque))

    # Convertir fecha al formato correcto
    fecha_iso = convertir_fecha(row[0])

    # Agregar propiedades con tipos de datos adecuados
    g.add((registro_uri, EX.fecha, Literal(fecha_iso, datatype=XSD.date)))  # Fecha corregida
    g.add((registro_uri, EX.año, Literal(int(row[1]), datatype=XSD.integer)))
    g.add((registro_uri, EX.mes, Literal(int(row[2]), datatype=XSD.integer)))
    g.add((registro_uri, EX.dia, Literal(int(row[3]), datatype=XSD.integer)))
    g.add((registro_uri, EX.bloqueHorario, Literal(int(row[4]), datatype=XSD.integer)))
    g.add((registro_uri, EX.ruta, Literal(row[5], datatype=XSD.string)))
    g.add((registro_uri, EX.cantidadPasajeros, Literal(int(row[6]), datatype=XSD.integer)))

# Guardar el grafo en formato Turtle
rdf_filename = "embarques_aeropuerto.ttl"
g.serialize(destination=rdf_filename, format="turtle")

# Descargar el archivo generado
files.download(rdf_filename)

print(f"Archivo RDF '{rdf_filename}' generado y listo para descargar.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivo RDF 'embarques_aeropuerto.ttl' generado y listo para descargar.


## **Segundo Conjunto de Datos (XLSX):**"Vehículos registrados en el Registro Civil por medio del Registro de Vehículos Motorizados."

## Script, Generación en formato RDF, archivo ttl

In [ ]:
# 📍 Instalar dependencias necesarias
!pip install rdflib pandas openpyxl

In [ ]:
import pandas as pd
import rdflib
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD
from google.colab import files

# 📌 Ruta del archivo XLSX en Google Colab
xlsx_filepath = "/content/sample_data/C026Inscripciondevehiculos_2927501738358921050.xlsx"

# 📌 Cargar el archivo Excel en un DataFrame de Pandas
df = pd.read_excel(xlsx_filepath)

# 📌 Crear un grafo RDF
g = Graph()

# 📌 Definir un namespace personalizado
EX = Namespace("http://example.org/vehicles#")
g.bind("ex", EX)

# 📌 Convertir el DataFrame a RDF
for i, row in df.iterrows():
    registro_uri = URIRef(EX[f"registro_{i}"])

    # Agregar tipo de entidad
    g.add((registro_uri, RDF.type, EX.InscripcionVehiculo))

    # Iterar sobre las columnas y agregar propiedades al grafo
    for col in df.columns:
        valor = row[col]
        if pd.notna(valor):  # Verifica que no sea NaN
            prop_uri = URIRef(EX[col.replace(" ", "_")])  # Convertir espacios en _

            # Definir tipo de datos RDF
            if isinstance(valor, int):
                g.add((registro_uri, prop_uri, Literal(valor, datatype=XSD.integer)))
            elif isinstance(valor, float):
                g.add((registro_uri, prop_uri, Literal(valor, datatype=XSD.decimal)))
            else:
                g.add((registro_uri, prop_uri, Literal(str(valor), datatype=XSD.string)))

# 📌 Guardar el grafo en formato Turtle
rdf_filename = "inscripcion_vehiculos.ttl"
g.serialize(destination=rdf_filename, format="turtle")

# 📌 Descargar el archivo generado
files.download(rdf_filename)

print(f"✅ Archivo RDF '{rdf_filename}' generado y listo para descargar.")

# **Tarea 2/Opcional 2**: Consultas SPARQL. Datos en Memoria

## **Primer Conjunto de Datos (JSON):** "Cantidad de pasajeros embarcados por bloque horario en el Aeropuerto de Punta Arenas Presidente Carlos Ibáñez del Campo"

* Consulta 1: Registros de embarque del 20 de abril de 2019
* Consulta 2: Cantidad total de pasajeros embarcados en abril
* Consulta 3: Registros con más de 200 pasajeros embarcados
* Consulta 4: Cantidad de vuelos nacionales vs. internacionales

In [ ]:
import rdflib
from rdflib import Graph, Namespace
from rdflib.plugins.sparql import prepareQuery
import pandas as pd

# 📍 Ruta del archivo RDF almacenado en Google Colab
rdf_filepath = "/content/embarques_aeropuerto.ttl"

# 📍 Crear un grafo RDF y cargar los datos desde el archivo Turtle
g = Graph()
g.parse(rdf_filepath, format="turtle")

# 📍 Definir el namespace
EX = Namespace("http://example.org/airport#")

# 🔍 **Consulta 1: Registros de embarque del 20 de abril de 2019**
query_1 = prepareQuery("""
    SELECT ?registro ?fecha ?cantidadPasajeros WHERE {
        ?registro a ex:RegistroEmbarque ;
                  ex:fecha "2019-04-20"^^xsd:date ;
                  ex:cantidadPasajeros ?cantidadPasajeros .
    }
""", initNs={"ex": EX})

# 🔍 **Consulta 2: Cantidad total de pasajeros embarcados en abril**
query_2 = prepareQuery("""
    SELECT ?mes (SUM(?cantidadPasajeros) AS ?totalPasajeros) WHERE {
        ?registro a ex:RegistroEmbarque ;
                  ex:mes ?mes ;
                  ex:cantidadPasajeros ?cantidadPasajeros .
        FILTER(?mes = 4)
    }
    GROUP BY ?mes
""", initNs={"ex": EX})

# 🔍 **Consulta 3: Registros con más de 200 pasajeros embarcados**
query_3 = prepareQuery("""
    SELECT ?registro ?fecha ?cantidadPasajeros WHERE {
        ?registro a ex:RegistroEmbarque ;
                  ex:fecha ?fecha ;
                  ex:cantidadPasajeros ?cantidadPasajeros .
        FILTER(?cantidadPasajeros > 200)
    }
    ORDER BY DESC(?cantidadPasajeros)
""", initNs={"ex": EX})

# 🔍 **Consulta 4: Cantidad de vuelos nacionales vs. internacionales**
query_4 = prepareQuery("""
    SELECT ?ruta (COUNT(?registro) AS ?totalVuelos) WHERE {
        ?registro a ex:RegistroEmbarque ;
                  ex:ruta ?ruta .
    }
    GROUP BY ?ruta
""", initNs={"ex": EX})

# 📍 **Ejecutar consultas**
results_1 = list(g.query(query_1))
results_2 = list(g.query(query_2))
results_3 = list(g.query(query_3))
results_4 = list(g.query(query_4))

# 📍 **Crear DataFrames para visualizar los resultados**
df_1 = pd.DataFrame(results_1, columns=["Registro", "Fecha", "Cantidad Pasajeros"])
df_2 = pd.DataFrame(results_2, columns=["Mes", "Total Pasajeros"])
df_3 = pd.DataFrame(results_3, columns=["Registro", "Fecha", "Cantidad Pasajeros"])
df_4 = pd.DataFrame(results_4, columns=["Ruta", "Total Vuelos"])

# 📍 **Mostrar resultados**
print("\n📌 Registros de embarque del 20 de abril de 2019:")
print(df_1.head())

print("\n📌 Cantidad total de pasajeros embarcados en abril:")
print(df_2.head())

print("\n📌 Registros de embarque con más de 200 pasajeros:")
print(df_3.head())

print("\n📌 Cantidad de vuelos nacionales e internacionales:")
print(df_4.head())



📌 Registros de embarque del 20 de abril de 2019:
                                 Registro Fecha Cantidad Pasajeros
0  http://example.org/airport#registro_10  None                151
1  http://example.org/airport#registro_11  None                118
2  http://example.org/airport#registro_12  None                153
3  http://example.org/airport#registro_13  None                194
4   http://example.org/airport#registro_2  None                189

📌 Cantidad total de pasajeros embarcados en abril:
  Mes Total Pasajeros
0   4          100206

📌 Registros de embarque con más de 200 pasajeros:
                                   Registro       Fecha Cantidad Pasajeros
0  http://example.org/airport#registro_7635  2022-02-15                716
1  http://example.org/airport#registro_7498  2022-02-04                629
2  http://example.org/airport#registro_3025  2020-01-07                621
3  http://example.org/airport#registro_9127  2022-08-15                619
4  http://example.org/airp

## **Segundo Conjunto de Datos (XML):**"Vehículos registrados en el Registro Civil por medio del Registro de Vehículos Motorizados, esta información es consolidad y disponibilizada por ANAC A.G"




* ¿Cuáles son los tipos de vehículos que más se venden en enero?
* ¿Cuál es el tipo de vehículo que más se vende en general?
* ¿Cuál es el tipo de vehículo que más se vendió en 2020?
* ¿Cuántos tipos diferentes de vehículos hay?




In [ ]:
# 📍 Instalar rdflib y pandas si no están instalados
!pip install rdflib pandas

import rdflib
import pandas as pd
from rdflib import Graph

# 📌 Ruta del archivo RDF en formato Turtle
rdf_filepath = "/content/inscripcion_vehiculos.ttl"

# 📌 Cargar el archivo RDF en un grafo
g = Graph()
g.parse(rdf_filepath, format="turtle")

# 📌 Función para ejecutar consultas SPARQL y mostrar resultados
def ejecutar_consulta(sparql_query, descripcion):
    print(f"\n🔹 {descripcion}\n")
    qres = g.query(sparql_query)

    # Convertir resultados en DataFrame para mejor visualización
    resultados = []
    for row in qres:
        resultados.append([str(valor) for valor in row])

    if resultados:
        df = pd.DataFrame(resultados, columns=[str(var) for var in qres.vars])
        from google.colab import data_table
        display(df)  # 📌 Muestra la tabla en Colab
    else:
        print("⚠️ No se encontraron resultados.")

# 📌 Consultas SPARQL corregidas:

# 1️⃣ ¿Cuáles son los tipos de vehículos que más se venden en enero?
consulta1 = """
PREFIX ex: <http://example.org/vehicles#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?tipo (SUM(?total_cantidad) AS ?total_vendidos)
WHERE {
  ?vehiculo ex:mes "Enero"^^xsd:string ;
            ex:tipo ?tipo ;
            ex:total_cantidad ?total_cantidad .
}
GROUP BY ?tipo
ORDER BY DESC(?total_vendidos)
"""
ejecutar_consulta(consulta1, "📌 Tipos de vehículos más vendidos en Enero")

# 2️⃣ ¿Cuál es el tipo de vehículo que más se vende en general?
consulta2 = """
PREFIX ex: <http://example.org/vehicles#>

SELECT ?tipo (SUM(?total_cantidad) AS ?total_vendidos)
WHERE {
  ?vehiculo ex:tipo ?tipo ;
            ex:total_cantidad ?total_cantidad .
}
GROUP BY ?tipo
ORDER BY DESC(?total_vendidos)
LIMIT 1
"""
ejecutar_consulta(consulta2, "📌 Tipo de vehículo más vendido")

# 3️⃣ ¿Cuál es el tipo de vehículo que más se vendió en 2020?
consulta3 = """
PREFIX ex: <http://example.org/vehicles#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?tipo (SUM(?total_cantidad) AS ?total_vendidos)
WHERE {
  ?vehiculo ex:anio 2020 ;
            ex:tipo ?tipo ;
            ex:total_cantidad ?total_cantidad .
}
GROUP BY ?tipo
ORDER BY DESC(?total_vendidos)
LIMIT 1
"""
ejecutar_consulta(consulta3, "📌 Tipo de vehículo más vendido en 2020")

# 4️⃣ ¿Cuántos tipos diferentes de vehículos hay?
consulta4 = """
PREFIX ex: <http://example.org/vehicles#>

SELECT (COUNT(DISTINCT ?tipo) AS ?total_tipos)
WHERE {
  ?vehiculo ex:tipo ?tipo .
}
"""
ejecutar_consulta(consulta4, "📌 Cantidad de tipos diferentes de vehículos")



🔹 📌 Tipos de vehículos más vendidos en Enero



,tipo,total_vendidos
0,Otros,243861
1,Vehículo Comercial,27885
2,Camión,6677
3,Tractocamión,3335
4,Chasis Cabina,545
5,Carro Bomba,161



🔹 📌 Tipo de vehículo más vendido



,tipo,total_vendidos
0,Otros,2858937



🔹 📌 Tipo de vehículo más vendido en 2020



,tipo,total_vendidos
0,Otros,232368



🔹 📌 Cantidad de tipos diferentes de vehículos



,total_tipos
0,7


# **Tarea 3/Opcional 3:**  Crear un modelo de datos en Shape Expressions (ShEx) y  SHACL que permita validar los datos. Compararlos

## **Primer Conjunto de Datos (JSON):** "Cantidad de pasajeros embarcados por bloque horario en el Aeropuerto de Punta Arenas Presidente Carlos Ibáñez del Campo"

### SHACL

In [ ]:
# 📍 Instalar pyshacl si no está instalado
!pip install pyshacl rdflib

import rdflib
from pyshacl import validate
import logging

# 📍 Desactivar todos los mensajes excepto errores
logging.getLogger("pyshacl").setLevel(logging.ERROR)
logging.getLogger("rdflib").setLevel(logging.ERROR)

# 📍 Cargar datos RDF
rdf_filepath = "/content/embarques_aeropuerto.ttl"
rdf_graph = rdflib.Graph()
rdf_graph.parse(rdf_filepath, format="turtle")

# 📍 Definir el modelo SHACL
shacl_ttl = """
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix ex: <http://example.org/airport#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:RegistroEmbarqueShape
    a sh:NodeShape ;
    sh:targetClass ex:RegistroEmbarque ;

    # 📍 Restricción para "fecha"
    sh:property [
        sh:path ex:fecha ;
        sh:datatype xsd:date ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:description "La fecha del embarque debe estar en formato YYYY-MM-DD."@es ;
    ] ;

    # 📍 Restricción para "año"
    sh:property [
        sh:path ex:año ;
        sh:datatype xsd:integer ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:minInclusive 1900 ;
        sh:maxInclusive 2100 ;
        sh:description "El año debe ser un número entero entre 1900 y 2100."@es ;
    ] ;

    # 📍 Restricción para "mes"
    sh:property [
        sh:path ex:mes ;
        sh:datatype xsd:integer ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:minInclusive 1 ;
        sh:maxInclusive 12 ;
        sh:description "El mes debe ser un número entero entre 1 y 12."@es ;
    ] ;

    # 📍 Restricción para "día"
    sh:property [
        sh:path ex:dia ;
        sh:datatype xsd:integer ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:minInclusive 1 ;
        sh:maxInclusive 31 ;
        sh:description "El día debe ser un número entero entre 1 y 31."@es ;
    ] ;

    # 📍 Restricción para "bloqueHorario"
    sh:property [
        sh:path ex:bloqueHorario ;
        sh:datatype xsd:integer ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:minInclusive 0 ;
        sh:maxInclusive 23 ;
        sh:description "El bloque horario debe ser un número entero entre 0 y 23."@es ;
    ] ;

    # 📍 Restricción para "ruta"
    sh:property [
        sh:path ex:ruta ;
        sh:datatype xsd:string ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:in ( "NACIONAL"^^xsd:string "INTERNACIONAL"^^xsd:string ) ;
        sh:description "La ruta del embarque debe ser 'NACIONAL' o 'INTERNACIONAL'."@es ;
    ] ;

    # 📍 Restricción para "cantidadPasajeros"
    sh:property [
        sh:path ex:cantidadPasajeros ;
        sh:datatype xsd:integer ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:minInclusive 0 ;
        sh:description "La cantidad de pasajeros debe ser un número entero 0 o mayor."@es ;
    ] .
"""

# 📍 Cargar el modelo SHACL en un grafo
shacl_graph = rdflib.Graph()
shacl_graph.parse(data=shacl_ttl, format="turtle")

# 📍 Validar solo los primeros 10 registros
print("\n📌 Iniciando validación de los primeros 10 registros...\n")

count_valid = 0
count_invalid = 0

for i, registro in enumerate(rdf_graph.subjects(rdflib.RDF.type, rdflib.URIRef("http://example.org/airport#RegistroEmbarque"))):

    # Crear un subgrafo RDF con solo este registro
    registro_graph = rdflib.Graph()

    # Agregar las triples de este registro al subgrafo
    for triple in rdf_graph.triples((registro, None, None)):
        registro_graph.add(triple)

    # Ejecutar la validación SHACL solo para este registro
    conforms, results_graph, results_text = validate(registro_graph, shacl_graph=shacl_graph, inference="rdfs")

    # 📍 Mostrar resultado de la validación para cada registro
    if conforms:
        print(f"✅ Registro {i+1}: VÁLIDO → {registro}")
        count_valid += 1
    else:
        print(f"❌ Registro {i+1}: INVÁLIDO → {registro}")
        print("   🔹 Detalles del error:")
        print(results_text)
        count_invalid += 1

    # 📍 Limitar la validación a 10 registros para evitar demoras
    if i >= 9:
        print("\n⚠️ Se han validado 10 registros como muestra. Deteniendo validación para optimizar tiempo.\n")
        break

# 📍 Resumen final de la validación
print("\n📌 Resumen de la validación:")
print(f"✅ Registros válidos: {count_valid}")
print(f"❌ Registros inválidos: {count_invalid}")



  Using cached pyshacl-0.30.0-py3-none-any.whl.metadata (35 kB)
  Using cached owlrl-7.1.3-py3-none-any.whl.metadata (3.6 kB)
  Using cached html5rdf-1.2.1-py2.py3-none-any.whl.metadata (7.5 kB)
Using cached pyshacl-0.30.0-py3-none-any.whl (1.3 MB)
Using cached owlrl-7.1.3-py3-none-any.whl (51 kB)
Using cached html5rdf-1.2.1-py2.py3-none-any.whl (109 kB)

📌 Iniciando validación de los primeros 10 registros...

✅ Registro 1: VÁLIDO → http://example.org/airport#registro_0
✅ Registro 2: VÁLIDO → http://example.org/airport#registro_1
✅ Registro 3: VÁLIDO → http://example.org/airport#registro_10
✅ Registro 4: VÁLIDO → http://example.org/airport#registro_100
✅ Registro 5: VÁLIDO → http://example.org/airport#registro_1000
✅ Registro 6: VÁLIDO → http://example.org/airport#registro_1001
✅ Registro 7: VÁLIDO → http://example.org/airport#registro_1002
✅ Registro 8: VÁLIDO → http://example.org/airport#registro_1003
✅ Registro 9: VÁLIDO → http://example.org/airport#registro_1004
✅ Registro 10: VÁLI

### ShEx

In [ ]:
# 📍 Instalar pyshex si no está instalado
!pip install pyshex rdflib

import rdflib
from pyshex import ShExEvaluator
import logging

# 📍 Desactivar mensajes de debug y warnings
logging.getLogger("rdflib").setLevel(logging.ERROR)
logging.getLogger("pyshex").setLevel(logging.ERROR)

# 📍 Definir el modelo ShEx corregido (PERMITIENDO cantidadPasajeros = 0)
shex_schema = """
PREFIX ex: <http://example.org/airport#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

start = @ex:RegistroEmbarqueShape  # 🔹 Asegura que inicie con esta forma

ex:RegistroEmbarqueShape {
    ex:fecha xsd:date ;   # Fecha en formato YYYY-MM-DD
    ex:año xsd:integer MinInclusive 1900 MaxInclusive 2100 ;  # Año entre 1900 y 2100
    ex:mes xsd:integer MinInclusive 1 MaxInclusive 12 ;  # Mes entre 1 y 12
    ex:dia xsd:integer MinInclusive 1 MaxInclusive 31 ;  # Día entre 1 y 31
    ex:bloqueHorario xsd:integer MinInclusive 0 MaxInclusive 23 ;  # Horas entre 0 y 23
    ex:cantidadPasajeros xsd:integer MinInclusive 0  # ⚠️ Ahora acepta 0 pasajeros
}
"""

# 📍 Cargar los datos RDF
rdf_filepath = "/content/embarques_aeropuerto.ttl"
rdf_graph = rdflib.Graph()
rdf_graph.parse(rdf_filepath, format="turtle")

# 📍 Mostrar el contenido del primer registro RDF para diagnóstico
print("\n📌 Primer registro RDF detectado:\n")
for s in rdf_graph.subjects(rdflib.RDF.type, rdflib.URIRef("http://example.org/airport#RegistroEmbarque")):
    print(f"Registro: {s}")

    # Imprimir todas las propiedades de este registro
    for p, o in rdf_graph.predicate_objects(subject=s):
        print(f"   🔹 {p} → {o}")

    break  # Solo mostramos el primer registro

# 📍 Inicializar el validador ShEx con el esquema corregido
evaluator = ShExEvaluator(rdf_graph, shex_schema)

# 📍 Validar solo los primeros 10 registros
print("\n📌 Iniciando validación con ShEx de los primeros 10 registros...\n")

count_valid = 0
count_invalid = 0

for i, registro in enumerate(rdf_graph.subjects(rdflib.RDF.type, rdflib.URIRef("http://example.org/airport#RegistroEmbarque"))):

    # 📍 Evaluar este registro con ShEx
    results = evaluator.evaluate(focus=str(registro))

    # 📍 Mostrar resultado de la validación para cada registro
    if results[0].result:
        print(f"✅ Registro {i+1}: VÁLIDO → {registro}")
        count_valid += 1
    else:
        print(f"\n❌ Registro {i+1}: INVÁLIDO → {registro}")
        for r in results:
            print(f"🔹 Nodo evaluado: {r.focus}")
            print(f"🔹 Mensaje de error: {r.reason}")
        count_invalid += 1

    # 📍 Limitar la validación a 10 registros
    if i >= 9:
        print("\n⚠️ Se han validado 10 registros como muestra. Deteniendo validación para optimizar tiempo.\n")
        break

# 📍 Resumen final de la validación
print("\n📌 Resumen de la validación:")
print(f"✅ Registros válidos: {count_valid}")
print(f"❌ Registros inválidos: {count_invalid}")


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

## **Segundo Conjunto de Datos (XML):**"Vehículos registrados en el Registro Civil por medio del Registro de Vehículos Motorizados, esta información es consolidad y disponibilizada por ANAC A.G"

### SHACL

In [ ]:
# 📍 Instalar dependencias necesarias
!pip install pyshacl rdflib

import rdflib
from pyshacl import validate
import logging

# 📍 Desactivar logs innecesarios
logging.getLogger("pyshacl").setLevel(logging.ERROR)
logging.getLogger("rdflib").setLevel(logging.ERROR)

# 📍 Cargar los datos RDF
rdf_filepath = "/content/inscripcion_vehiculos.ttl"
rdf_graph = rdflib.Graph()
rdf_graph.parse(rdf_filepath, format="turtle")

# 📍 Definir el modelo SHACL
shacl_ttl = """
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix ex: <http://example.org/vehicles#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:InscripcionVehiculoShape
    a sh:NodeShape ;
    sh:targetClass ex:InscripcionVehiculo ;

    sh:property [
        sh:path ex:anio ;
        sh:datatype xsd:integer ;
        sh:minInclusive 1900 ;
        sh:maxInclusive 2100 ;
    ] ;

    sh:property [
        sh:path ex:mes ;
        sh:datatype xsd:string ;
    ] ;

    sh:property [
        sh:path ex:tipo ;
        sh:datatype xsd:string ;
    ] ;

    sh:property [
        sh:path ex:total_cantidad ;
        sh:datatype xsd:integer ;
        sh:minInclusive 0 ;
    ] .
"""

# 📍 Cargar el modelo SHACL
shacl_graph = rdflib.Graph()
shacl_graph.parse(data=shacl_ttl, format="turtle")

# 📍 Validar solo los primeros 10 registros
print("\n📌 Iniciando validación SHACL de los primeros 10 registros...\n")

count_valid = 0
count_invalid = 0

for i, registro in enumerate(rdf_graph.subjects(rdflib.RDF.type, rdflib.URIRef("http://example.org/vehicles#InscripcionVehiculo"))):

    if i >= 10:  # 📌 Limita la validación a 10 registros
        break

    # Crear un subgrafo RDF con solo este registro
    registro_graph = rdflib.Graph()
    for triple in rdf_graph.triples((registro, None, None)):
        registro_graph.add(triple)

    # Ejecutar la validación SHACL solo para este registro
    conforms, results_graph, results_text = validate(registro_graph, shacl_graph=shacl_graph, inference="rdfs")

    # 📍 Mostrar resultado de la validación para cada registro
    print(f"\n🔹 Validando registro {i+1}: {registro}")
    if conforms:
        print(f"✅ Registro {i+1}: VÁLIDO")
        count_valid += 1
    else:
        print(f"❌ Registro {i+1}: INVÁLIDO")
        print("   🔹 Detalles del error:\n", results_text)
        count_invalid += 1

# 📍 Resumen final de la validación
print("\n📌 Resumen de la validación SHACL:")
print(f"✅ Registros válidos: {count_valid}")
print(f"❌ Registros inválidos: {count_invalid}")


📌 Iniciando validación SHACL de los primeros 10 registros...


🔹 Validando registro 1: http://example.org/vehicles#registro_0
✅ Registro 1: VÁLIDO

🔹 Validando registro 2: http://example.org/vehicles#registro_1
✅ Registro 2: VÁLIDO

🔹 Validando registro 3: http://example.org/vehicles#registro_10
✅ Registro 3: VÁLIDO

🔹 Validando registro 4: http://example.org/vehicles#registro_100
✅ Registro 4: VÁLIDO

🔹 Validando registro 5: http://example.org/vehicles#registro_101
✅ Registro 5: VÁLIDO

🔹 Validando registro 6: http://example.org/vehicles#registro_102
✅ Registro 6: VÁLIDO

🔹 Validando registro 7: http://example.org/vehicles#registro_103
✅ Registro 7: VÁLIDO

🔹 Validando registro 8: http://example.org/vehicles#registro_104
✅ Registro 8: VÁLIDO

🔹 Validando registro 9: http://example.org/vehicles#registro_105
✅ Registro 9: VÁLIDO

🔹 Validando registro 10: http://example.org/vehicles#registro_106
✅ Registro 10: VÁLIDO

📌 Resumen de la validación SHACL:
✅ Registros válidos: 10
❌ Registros

### ShEX

In [ ]:
# 📍 Instalar dependencias necesarias
!pip install pyshex rdflib

import rdflib
from pyshex import ShExEvaluator
import logging

# 📍 Desactivar logs innecesarios
logging.getLogger("rdflib").setLevel(logging.ERROR)
logging.getLogger("pyshex").setLevel(logging.ERROR)

# 📍 Definir el modelo ShEx
shex_schema = """
PREFIX ex: <http://example.org/vehicles#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

start = @ex:InscripcionVehiculoShape

ex:InscripcionVehiculoShape {
    ex:anio xsd:integer MinInclusive 1900 MaxInclusive 2100 ;
    ex:mes xsd:string ;
    ex:tipo xsd:string ;
    ex:total_cantidad xsd:integer MinInclusive 0 ;
}
"""

# 📍 Cargar los datos RDF
rdf_filepath = "/content/inscripcion_vehiculos.ttl"
rdf_graph = rdflib.Graph()
rdf_graph.parse(rdf_filepath, format="turtle")

# 📍 Inicializar el validador ShEx con el esquema
evaluator = ShExEvaluator(rdf_graph, shex_schema)

# 📍 Validar solo los primeros 10 registros
print("\n📌 Iniciando validación con ShEx de los primeros 10 registros...\n")

count_valid = 0
count_invalid = 0

for i, registro in enumerate(rdf_graph.subjects(rdflib.RDF.type, rdflib.URIRef("http://example.org/vehicles#InscripcionVehiculo"))):

    if i >= 10:  # 📌 Limita la validación a 10 registros
        break

    # 📍 Evaluar este registro con ShEx
    results = evaluator.evaluate(focus=str(registro))

    # 📍 Mostrar resultado de la validación para cada registro
    print(f"\n🔹 Validando registro {i+1}: {registro}")
    if results[0].result:
        print(f"✅ Registro {i+1}: VÁLIDO")
        count_valid += 1
    else:
        print(f"❌ Registro {i+1}: INVÁLIDO")
        for r in results:
            print(f"   🔹 Nodo evaluado: {r.focus}")
            print(f"   🔹 Mensaje de error: {r.reason}")
        count_invalid += 1

# 📍 Resumen final de la validación
print("\n📌 Resumen de la validación ShEx:")
print(f"✅ Registros válidos: {count_valid}")
print(f"❌ Registros inválidos: {count_invalid}")



📌 Iniciando validación con ShEx de los primeros 10 registros...


🔹 Validando registro 1: http://example.org/vehicles#registro_0
✅ Registro 1: VÁLIDO

🔹 Validando registro 2: http://example.org/vehicles#registro_1
✅ Registro 2: VÁLIDO

🔹 Validando registro 3: http://example.org/vehicles#registro_10
✅ Registro 3: VÁLIDO

🔹 Validando registro 4: http://example.org/vehicles#registro_100
✅ Registro 4: VÁLIDO

🔹 Validando registro 5: http://example.org/vehicles#registro_101
✅ Registro 5: VÁLIDO

🔹 Validando registro 6: http://example.org/vehicles#registro_102
✅ Registro 6: VÁLIDO

🔹 Validando registro 7: http://example.org/vehicles#registro_103
✅ Registro 7: VÁLIDO

🔹 Validando registro 8: http://example.org/vehicles#registro_104
✅ Registro 8: VÁLIDO

🔹 Validando registro 9: http://example.org/vehicles#registro_105
✅ Registro 9: VÁLIDO

🔹 Validando registro 10: http://example.org/vehicles#registro_106
✅ Registro 10: VÁLIDO

📌 Resumen de la validación ShEx:
✅ Registros válidos: 10
❌ Registr

## **Comparación entre SHACL y ShEx en la validación de datos RDF**

1. Propósito y Enfoque
SHACL es más flexible y poderoso, pero más verboso.
ShEx es más sencillo y directo, ideal para validaciones rápidas.

2. Sintaxis y Facilidad de Uso
ShEx es más fácil de leer y escribir, pero menos flexible.
SHACL es más detallado y robusto, ideal para reglas avanzadas.

3. Expresividad y Validación
SHACL es más poderoso para definir reglas complejas.
ShEx es más simple y fácil de aplicar, pero menos expresivo.

4. Rendimiento y Escalabilidad
SHACL es mejor para bases de datos grandes.
ShEx es más rápido en validaciones sencillas.

5. Aplicaciones y Casos de Uso
SHACL es ideal para entornos empresariales y validaciones complejas.
ShEx es más útil en validaciones rápidas y validaciones estructurales simples.

# **Tarea 4/Opcional 4:** Comparativa entre RDF con SPARQL, Wikidata y LLMs.

Compararemos las respuestas obtenidas de diferentes fuentes de datos sobre inscripción de vehículos. Se evaluará la calidad de las respuestas obtenidas desde:

* RDF + SPARQL (Datos estructurados en un grafo RDF con
 consultas SPARQL)
* Wikidata (Base de conocimiento colaborativa con consultas SPARQL)
* Large Language Models (LLMs) como GPT-4 (Generación de respuestas basadas en aprendizaje profundo)

# **1. Comparación de respuestas entre RDF + SPARQL, Wikidata y LLMs**

Elegimos las siguientes preguntas para la comparación:

* Pregunta 1: ¿Cuál fue el tipo de vehículo más vendido en 2020?
* Pregunta 2: ¿Cuántos tipos diferentes de vehículos se registraron en el dataset?

# **Respuesta mediante RDF + SPARQL (sobre nuestros datos)**

In [ ]:
# 📍 Instalar rdflib y pandas si no están instalados
!pip install rdflib pandas

import rdflib
import pandas as pd

# 📌 Cargar el archivo RDF
rdf_filepath = "/content/inscripcion_vehiculos.ttl"  # 📌 Ruta del archivo RDF
g = rdflib.Graph()
g.parse(rdf_filepath, format="turtle")

# 📌 Función para ejecutar consultas SPARQL y mostrar resultados
def ejecutar_consulta(sparql_query, titulo):
    print(f"\n🔹 {titulo}\n")
    qres = g.query(sparql_query)

    # Convertir resultados en DataFrame
    resultados = []
    for row in qres:
        resultados.append([str(valor) for valor in row])

    if resultados:
        df = pd.DataFrame(resultados, columns=[str(var) for var in qres.vars])
        display(df)  # 📌 Muestra la tabla en Colab
    else:
        print("⚠️ No se encontraron resultados.")

# 📌 Pregunta 1: ¿Cuál fue el tipo de vehículo más vendido en 2020?
consulta1 = """
PREFIX ex: <http://example.org/vehicles#>

SELECT ?tipo (SUM(xsd:integer(?total_cantidad)) AS ?total_vendidos)
WHERE {
  ?vehiculo ex:anio 2020 ;
            ex:tipo ?tipo ;
            ex:total_cantidad ?total_cantidad .
}
GROUP BY ?tipo
ORDER BY DESC(?total_vendidos)
LIMIT 1
"""
ejecutar_consulta(consulta1, "📌 Tipo de vehículo más vendido en 2020")

# 📌 Pregunta 2: ¿Cuántos tipos diferentes de vehículos se registraron en el dataset?
consulta2 = """
PREFIX ex: <http://example.org/vehicles#>

SELECT (COUNT(DISTINCT ?tipo) AS ?total_tipos)
WHERE {
  ?vehiculo ex:tipo ?tipo .
}
"""
ejecutar_consulta(consulta2, "📌 Cantidad de tipos diferentes de vehículos registrados")



🔹 📌 Tipo de vehículo más vendido en 2020



,tipo,total_vendidos
0,Otros,232368



🔹 📌 Cantidad de tipos diferentes de vehículos registrados



,total_tipos
0,7


# **Fortalezas de RDF + SPARQL:**
* Consultas exactas y estructuradas
* Alta precisión en los resultados
* Datos completamente verificables y auditable

# **Desventajas de RDF + SPARQL:**
* Requiere conocimiento técnico en SPARQL
* No es accesible para usuarios sin experiencia en bases de datos

# **Respuesta mediante Wikidata**

In [ ]:
# 📍 Instalar librerías necesarias
!pip install SPARQLWrapper pandas

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

# 📌 Configurar el endpoint de Wikidata
endpoint_url = "https://query.wikidata.org/sparql"

# 📌 Función para ejecutar consultas SPARQL en Wikidata
def ejecutar_consulta_wikidata(sparql_query, titulo):
    print(f"\n🔹 {titulo}\n")
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    # Convertir los resultados a un DataFrame
    datos = []
    for result in results["results"]["bindings"]:
        fila = {var: result[var]["value"] for var in result}
        datos.append(fila)

    if datos:
        df = pd.DataFrame(datos)
        display(df)  # 📌 Muestra la tabla en Colab
    else:
        print("⚠️ No se encontraron resultados.")

# 📌 Pregunta 1: ¿Cuál fue el tipo de vehículo más vendido en 2020?
consulta1_wikidata = """
SELECT ?vehicleTypeLabel (SUM(?quantity) AS ?totalSold) WHERE {
  ?registration wdt:P31 wd:Q80730726 ;  # Instancia de inscripción de vehículos
               wdt:P571 ?date ;          # Fecha de inscripción
               wdt:P4003 ?vehicleType ;  # Tipo de vehículo
               wdt:P1114 ?quantity .     # Cantidad inscrita
  FILTER(YEAR(?date) = 2020)  # Filtrar solo año 2020
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?vehicleTypeLabel
ORDER BY DESC(?totalSold)
LIMIT 1
"""
ejecutar_consulta_wikidata(consulta1_wikidata, "📌 Tipo de vehículo más vendido en 2020 (Wikidata)")

# 📌 Pregunta 2: ¿Cuántos tipos diferentes de vehículos se registraron en el dataset?
consulta2_wikidata = """
SELECT (COUNT(DISTINCT ?vehicleType) AS ?totalTypes) WHERE {
  ?registration wdt:P31 wd:Q80730726 ;  # Instancia de inscripción de vehículos
               wdt:P4003 ?vehicleType . # Tipo de vehículo
}
"""
ejecutar_consulta_wikidata(consulta2_wikidata, "📌 Cantidad de tipos diferentes de vehículos registrados (Wikidata)")



🔹 📌 Tipo de vehículo más vendido en 2020 (Wikidata)

⚠️ No se encontraron resultados.

🔹 📌 Cantidad de tipos diferentes de vehículos registrados (Wikidata)



,totalTypes
0,0


# **Fortalezas de Wikidata:**
* Disponible públicamente y colaborativo
* Información más generalizada y accesible

# **Desventajas de Wikidata:**
* Puede haber datos incompletos o desactualizados
* No refleja datos específicos de nuestro dataset

# **Respuesta mediante LLMs (ChatGPT, GPT-4, etc.)**

* **Pregunta:** ¿Cuál fue el tipo de vehículo más vendido en 2020?

* **Respuesta:** En 2020, el tipo de vehículo más vendido a nivel mundial fue el automóvil compacto, con el Toyota Corolla liderando las ventas con 1,134,262 unidades.

# **Fortalezas de los LLMs:**
* Accesibles para cualquier usuario
* Pueden generar explicaciones comprensibles y contexto adicional

# **Problemas en las respuestas de los LLMs:**
* No acceden a datos estructurados en tiempo real
* Respuestas basadas en estimaciones y patrones de entrenamiento
* Imprecisiones y falta de verificabilidad